Package Installations

In [ ]:
!pip install kaggle

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 30.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.0 MB/s 
     |████████████████████████████████| 895 kB 50.5 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install -U torchtext==0.8.0

     |████████████████████████████████| 6.9 MB 12.6 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


Imports

In [ ]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
seed_val = 500
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Kaggle Dataset

In [ ]:
# Create the kaggle directory and read the uploaded kaggle.json file
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Download dataset
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

100% 41.0M/41.0M [00:01<00:00, 42.2MB/s]
100% 41.0M/41.0M [00:01<00:00, 34.3MB/s]


In [ ]:
# Unzip folder in Colab content folder
!unzip /content/fake-and-real-news-dataset.zip

Archive:  /content/fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [ ]:
df1 = pd.read_csv("Fake.csv")
df2 = pd.read_csv("True.csv")
df1["y"] = ["False"]*len(df1)
df2["y"] = ["True"]*len(df2)

In [ ]:
frames = [df1, df2]
corpus = pd.concat(frames, ignore_index=True)

In [ ]:
corpus["x"] = corpus["title"]

In [ ]:
y_encoder = LabelEncoder()
corpus['y'] = y_encoder.fit_transform(corpus['y'])

In [ ]:
corpus = corpus[["x", "y"]]

In [ ]:
corpus.drop_duplicates(inplace=True)
corpus

,x,y
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0
...,...,...
44892,North Korea shipments to Syria chemical arms a...,1
44894,LexisNexis withdrew two products from Chinese ...,1
44895,Minsk cultural hub becomes haven from authorities,1
44896,Vatican upbeat on possibility of Pope Francis ...,1


**Reddit Dataset**

In [ ]:
reddit_df = pd.read_csv('/content/news_posts.csv')
reddit_df

,Unnamed: 0,Title,Post Text,ID,Score,Total Comments,Post URL
0,0,People Are Accusing Robinhood Of Stealing From...,NaN,l7afyx,181017,4408,https://www.buzzfeednews.com/article/clarissaj...
1,1,US Military Could Lose Space Force Trademark t...,NaN,gyzw2p,129257,2844,https://www.cbr.com/us-military-lose-space-for...
2,2,White House threatens to fire anyone who tries...,NaN,jrskag,126468,4141,https://americanindependent.com/white-house-th...
3,3,Meta's threat to close down Facebook and Insta...,NaN,so0ree,126258,3786,https://www.cityam.com/metas-threat-to-close-d...
4,4,Don't eat or inject yourself with disinfectant...,NaN,g6zci5,125437,7024,https://www.cnn.com/world/live-news/coronaviru...
...,...,...,...,...,...,...,...
4898,987,Colorado pizza delivery driver saves mans life...,NaN,3b37gf,24221,898,http://www.postindependent.com/news/16943384-1...
4899,988,"Utah Woman Donates Over 1,200 Handmade Toys to...",NaN,90pbng,24216,888,https://www.insideedition.com/utah-woman-donat...
4900,989,Year of the Tiger marks increase in tiger popu...,NaN,sinyqg,24197,205,https://democratic-europe.eu/2022/02/01/%ef%bf...
4901,990,Florida cops deliver dresses made by a 99-year...,NaN,8q8njr,24164,183,http://www.miamiherald.com/news/state/florida/...


In [ ]:
reddit_df['x'] = reddit_df['Title']

In [ ]:
reddit_df['y'] = ["True"]*4903
reddit_df['y'] = y_encoder.transform(reddit_df['y'])

In [ ]:
reddit_df = reddit_df[["x", "y"]]
reddit_df.drop_duplicates(inplace=True)
reddit_df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y
0,People Are Accusing Robinhood Of Stealing From...,1
1,US Military Could Lose Space Force Trademark t...,1
2,White House threatens to fire anyone who tries...,1
3,Meta's threat to close down Facebook and Insta...,1
4,Don't eat or inject yourself with disinfectant...,1
...,...,...
4898,Colorado pizza delivery driver saves mans life...,1
4899,"Utah Woman Donates Over 1,200 Handmade Toys to...",1
4900,Year of the Tiger marks increase in tiger popu...,1
4901,Florida cops deliver dresses made by a 99-year...,1


Twitter Dataset

In [ ]:
twitter_df1 = pd.read_csv("/content/fake_news_tweets.csv")
twitter_df2 = pd.read_csv("/content/real_news_tweets.csv")
twitter_df1["y"] = ["False"]*len(twitter_df1)
twitter_df2["y"] = ["True"]*len(twitter_df2)

In [ ]:
twitter_df1['x'] = twitter_df1['Text']
twitter_df2['x'] = twitter_df2['Text']

In [ ]:
twitter_df = pd.concat([twitter_df1, twitter_df2], ignore_index=True)

In [ ]:
twitter_df['y'] = y_encoder.transform(twitter_df['y'])

In [ ]:
twitter_df = twitter_df[["x", "y"]]
twitter_df.drop_duplicates(inplace=True)
twitter_df

,x,y
0,Grandpa Surprisingly Willing To Talk About Man...,0
1,Nation Attempts To Fall Asleep By Doing Little...,0
2,"When it comes to pip blips, Megan Fox takes th...",0
3,Longtime Sleepytime Tea Addict Has To Use 6 Ba...,0
4,The Onion: Now on another part of your phone. ...,0
...,...,...
9995,The soldiers on the front line of the Ukraine-...,1
9996,What's Putin's next move?,1
9997,"Virgin Hyperloop, a futuristic train service e...",1
9998,Metaverse app allows kids into virtual strip c...,1


Data Preparation for Training

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
text = corpus.x.values
labels = corpus.y.values

In [ ]:
reddit_text = reddit_df.x.values
reddit_labels = reddit_df.y.values

In [ ]:
twitter_text = twitter_df.x.values
twitter_labels = twitter_df.y.values

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

In [ ]:
#Complete Tokenization
inputIds = [tokenizer.encode(element, add_special_tokens= True) for element in text]
redditInputIds = [tokenizer.encode(element, add_special_tokens= True) for element in reddit_text]
twitterInputIds = [tokenizer.encode(element, add_special_tokens= True) for element in twitter_text]

In [ ]:
#Truncating/Padding
MAX_LEN = 512
inputIdsTrunc = pad_sequences(inputIds, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
redditInputIdsTrunc = pad_sequences(redditInputIds, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
twitterInputIdsTrunc = pad_sequences(twitterInputIds, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [ ]:
#Attention Masks
attentionMasks = [[int(tokenId>0) for tokenId in ele] for ele in inputIdsTrunc]
redditAttentionMasks = [[int(tokenId>0) for tokenId in ele] for ele in redditInputIdsTrunc]
twitterAttentionMasks = [[int(tokenId>0) for tokenId in ele] for ele in twitterInputIdsTrunc]

In [ ]:
#Data-Type Conversion to Torch Tensor
trainInputs = torch.tensor(inputIdsTrunc)
redditInputs = torch.tensor(redditInputIdsTrunc)
twitterInputs = torch.tensor(twitterInputIdsTrunc)

trainLabels = torch.tensor(labels)
redditLabels = torch.tensor(reddit_labels)
twitterLabels = torch.tensor(twitter_labels)

trainMasks = torch.tensor(attentionMasks)
redditMasks = torch.tensor(redditAttentionMasks)
twitterMasks = torch.tensor(twitterAttentionMasks)

In [ ]:
batch_size = 4
#Train DataLoader
train_data = TensorDataset(trainInputs, trainMasks, trainLabels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
#Test DataLoader - Reddit
reddit_data = TensorDataset(redditInputs, redditMasks, redditLabels)
reddit_sampler = RandomSampler(reddit_data)
reddit_dataloader = DataLoader(reddit_data, sampler=reddit_sampler, batch_size=batch_size)
#Test DataLoader - Twitter
twitter_data = TensorDataset(twitterInputs, twitterMasks, twitterLabels)
twitter_sampler = RandomSampler(twitter_data)
twitter_dataloader = DataLoader(twitter_data, sampler=twitter_sampler, batch_size=batch_size)

Model Initialization

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels = 2)
model.cuda()

Downloading:   0%|          | 0.00/45.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [ ]:
#optimizer
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 2
train_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = train_steps)
scheduler

Training

In [ ]:
loss_values = []
for i in range(epochs):
    print("")
    print('Epoch: {}'.format(i + 1))
    print('Training...')
    # Reset the total loss for this epoch.
    total_loss = 0

    model.train()

    # For each batch of training data...
    for _, batch in enumerate(tqdm(train_dataloader)):
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 

        batch_input_ids = batch[0].to(device)
        batch_input_mask = batch[1].to(device)
        batch_labels = batch[2].to(device)
        model.zero_grad()        
        outputs = model(batch_input_ids, token_type_ids=None, attention_mask=batch_input_mask, labels=batch_labels)
        loss = outputs[0]
        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()
          
    loss_values.append(total_loss / len(train_dataloader) )

    print("")
    print("  Average training loss: {0:.2f}".format(total_loss/len(train_dataloader)))

print("")
print("Training complete!")


Epoch: 1
Training...


  0%|          | 0/9683 [00:00<?, ?it/s]


  Average training loss: 0.19

Epoch: 2
Training...


  0%|          | 0/9683 [00:00<?, ?it/s]


  Average training loss: 0.10

Training complete!


Prediction on Reddit Dataset

In [ ]:
predictions = []
true_labels = []
model.eval()
for batch in reddit_dataloader:
    batch = tuple(t.to(device) for t in batch)
    batchInputIds, batchInput_mask, batchLabels = batch
    with torch.no_grad():
        outputs = model(batchInputIds, token_type_ids=None, attention_mask=batchInput_mask)

    out = outputs[0]

    out = out.detach().cpu().numpy()
    labelIds = batchLabels.to('cpu').numpy()

    predictions.append(out)
    true_labels.append(labelIds)

In [ ]:
finalPredictions_reddit = [ele for predList in predictions for ele in predList]
finalPredictions_reddit = np.argmax(finalPredictions_reddit, axis=1).flatten()
finalTrueLabels_reddit = [ele for trueList in true_labels for ele in trueList]

Prediction on Twitter Dataset

In [ ]:
predictions = []
true_labels = []
model.eval()
for batch in twitter_dataloader:
    batch = tuple(t.to(device) for t in batch)
    batchInputIds, batchInput_mask, batchLabels = batch
    with torch.no_grad():
        outputs = model(batchInputIds, token_type_ids=None, attention_mask=batchInput_mask)

    out = outputs[0]

    out = out.detach().cpu().numpy()
    labelIds = batchLabels.to('cpu').numpy()

    predictions.append(out)
    true_labels.append(labelIds)

In [ ]:
finalPredictions_twitter = [ele for predList in predictions for ele in predList]
finalPredictions_twitter = np.argmax(finalPredictions_twitter, axis=1).flatten()
finalTrueLabels_twitter = [ele for trueList in true_labels for ele in trueList]

Results - Reddit

In [ ]:
accuracy_score(finalPredictions_reddit, finalTrueLabels_reddit)

0.513563124617581

In [ ]:
f1_score(finalPredictions_reddit, finalTrueLabels_reddit)

0.6786147419485243

Results - Twitter

In [ ]:
accuracy_score(finalPredictions_twitter, finalTrueLabels_twitter)

0.5941460132331502

In [ ]:
f1_score(finalPredictions_twitter, finalTrueLabels_twitter)

0.5560053981106612

Results - Twitter + Reddit

In [ ]:
accuracy_score(list(finalPredictions_reddit)+list(finalPredictions_twitter), list(finalTrueLabels_reddit)+list(finalTrueLabels_twitter))

0.5655571635311143

In [ ]:
f1_score(list(finalPredictions_reddit)+list(finalPredictions_twitter), list(finalTrueLabels_reddit)+list(finalTrueLabels_twitter))

0.6144361674800926

Saving Model

In [ ]:
torch.save(model.state_dict(), "albert_weights.pth")
torch.save(model, "albert.pth")